In [94]:
## Import Libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [95]:
## Data Sources:
# https://laurencemoroney.com/datasets.html

In [96]:
## Load data
train_path=r"C:\Users\JokeRR\Referances\python_projects\Rock_Paper_Scissor Classification\rps-test-set"
valid_path=r"C:\Users\JokeRR\Referances\python_projects\Rock_Paper_Scissor Classification\rps-test-set"

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen=ImageDataGenerator(rescale=1./255)
train=train_gen.flow_from_directory(directory=train_path,target_size=(224,224),batch_size=45)

valid_gen=ImageDataGenerator(rescale=1./255)
valid=valid_gen.flow_from_directory(directory=valid_path,target_size=(224,224),batch_size=45)

Found 372 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


In [97]:
## Class Mapper
labels=(train.class_indices)
labels

{'paper': 0, 'rock': 1, 'scissors': 2}

In [98]:
## Modeling

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

def CNN_model():
    ## create model
    model=Sequential()
    model.add(Conv2D(32,(3,3),strides=(1,1),activation='relu',input_shape=(224,224,3)))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2)))
    
    model.add(Conv2D(64,(5,5),strides=(1,1),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    
    model.add(Conv2D(64,(5,5),strides=(1,1),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    
    
    model.add(Flatten())
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(3,activation='softmax'))
    
    ## Model Compile
    model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

    
    return model
    

model=CNN_model()
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 107, 107, 64)      51264     
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 53, 53, 64)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 49, 49, 64)        102464    
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 24, 24, 64)        0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 36864)           

In [ ]:
## Early Stpping
callback=tf.keras.callbacks.EarlyStopping(monitor='loss',patience=3)

## Model Training
history=model.fit(x=train,steps_per_epoch=len(train),validation_data=valid,epochs=15,verbose=1,callbacks=[callback])

Epoch 1/15
9/9 [==============================] - 51s 6s/step - loss: 1.1439 - accuracy: 0.3199 - val_loss: 1.0662 - val_accuracy: 0.5753
Epoch 2/15
3/9 [=========>....................] - ETA: 21s - loss: 1.0717 - accuracy: 0.4593

In [ ]:
## Plotting Score & Loss Curve

acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']


## Plotting Accuracy Score
import matplotlib.pyplot as plt
plt.figure(figsize=(8,8))
#plt.subplot(1,2,1)
plt.plot(range(15),acc,label='Training Accuracy')
plt.plot(range(15),val_acc, label='Validation Accuracy')
plt.legend(loc='best')
plt.title('Accuracy')

In [ ]:
## Plotting Loss

#plt.subplot(1,2,2)
plt.plot(range(15),loss,label='Training Loss')
plt.plot(range(15),val_loss, label='Validation Loss')
plt.legend(loc='best')
plt.title('Loss')
plt.show()

In [37]:
## Prediction
pred=model.predict(x=test,steps=len(test),verbose=1)
pred

9/9 [==============================] - 9s 1s/step


array([[1.3498765e-05, 9.9997449e-01, 1.2042815e-05],
       [9.5333445e-01, 1.8132794e-05, 4.6647388e-02],
       [7.6378870e-05, 6.3806982e-04, 9.9928552e-01],
       ...,
       [9.9779415e-01, 2.5927720e-06, 2.2031975e-03],
       [2.2322542e-04, 9.9846590e-01, 1.3109993e-03],
       [3.3756576e-05, 3.4011342e-04, 9.9962616e-01]], dtype=float32)

In [89]:
## New Image Prediction
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

def predictor(img_path):
    img=image.load_img(img_path,target_size=(224,224))  # Image(PIL Image Instance)
    img_arr=image.img_to_array(img)                     # to Array(3D Array)
    img_expand=np.expand_dims(img_arr,axis=0)           # Expand Image-Array
    pred=model.predict(img_expand,batch_size=10)        # model Predict
    
    lst=["Paper","Rock","Scissors"]

    return img.show(),lst[pred.argmax()]

predictor(r"C:\Users\JokeRR\Referances\python_projects\Rock_Paper_Scissor Classification\rps-validation\paper-hires2.png")

(None, 'Paper')